This is the second of two data cleaning files to prepare the dumbarton oaks seals data for import to nodegoat. The documentation is not perfeclty consistent. Make sure to mount your drive. This notebook creates different CSV files at different points in the process, so you may start at different points without needed to run the entire notebook. Also, the variable names can be slightly frustrating.

In [ ]:
pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [ ]:
seals03 = pd.read_csv('sealsdata02.csv')

In [ ]:
seals03.head()

# Extracting Text from HTML in cells

In [ ]:
#EXTRACT TEXT FROM HTML IN SPREADSHEET

#Extracting text given Html content
def extract_text(html_content):
  '''str in HTML format -> str
  '''
  soup = BeautifulSoup(html_content, 'html.parser')
  paragraphs = soup.find_all('p')
  extracted_text = [p.get_text() for p in paragraphs]
  descriptions = soup.find_all('div', class_='description')
  for desc in descriptions:
      extracted_text.append(desc.get_text())
  return ' '.join(extracted_text)  # Join all extracted text into a single string

html_columns = ['obverse_description', 'reverse_description', 'commentary', 'previous_editions',
                'obverse_inscription',"obverse_transcription", 'reverse_inscription', 'reverse_transcription']
                #this does not include the inscription_in_english column because that also contained the inscription in greek
                #that column is processed later
#Apply function to dataset
for column in html_columns:
    seals03[column] = seals03[column].astype(str).apply(extract_text)
seals03[html_columns].head()

In [ ]:
#TESTING IF IT WILL BE POSSIBLE TO GET THE ENGLISH TEXT FROM THE HTML TAGS
#Originally, I tried to extract the english text from the processed text from the HTML which had no seperation between the greek and engish
#You can see this below under the heading "Getting English text Attempt 1(Failed)"

# Figuring out if the html has the appropriate number of paragraph tags, excluding empty tags.
def has_two_p_sections(html_content):
  if isinstance(html_content, str):
      soup = BeautifulSoup(html_content, 'html.parser')
      paragraphs = [p for p in soup.find_all('p') if p.get_text().strip()]
      if len(paragraphs) == 2:
          return True
  return False

# Understanding what rows without two p sections look like (This allowed me to update the function to discount empty tags, )
rows_without_two_p_sections = seals03[seals03['inscription_in_english'].apply(lambda x: not pd.isna(x) and not has_two_p_sections(x))]
rows_without_two_p_sections.drop(rows_without_two_p_sections.columns.difference(['inscription_in_english']), axis=1, inplace=True)
rows_without_two_p_sections.head()

<ipython-input-26-2ae237a2fcfb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_without_two_p_sections.drop(rows_without_two_p_sections.columns.difference(['inscription_in_english']), axis=1, inplace=True)


,inscription_in_english
567,<p>Θεοτόκε βοήθει τῷ σῷ δούλῳ Κωνσταντίνῳ μητρ...
735,<p>Τὴν πᾶσαν ἐλπίδα μου εἰς σὲ ἀνατίθημι. Θεοτ...
1047,"<p>Basil, autocrator</p>"
1077,"<p><i>Alexios Angelos, despotes.</i></p>"
1107,"<p><i>Andronikos Palaiologos, despotes</i>.</p>"


In [ ]:
# Function to test if a string has exactly two sections of <p> tags and return the text of the second non-empty <p> tag
def extract_second_p_text(html_content):
  if isinstance(html_content, str):
      soup = BeautifulSoup(html_content, 'html.parser')
      paragraphs = [p.get_text().strip() for p in soup.find_all('p') if p.get_text().strip()]
      if len(paragraphs) == 2:
          return paragraphs[1] if len(paragraphs) >= 2 else np.nan  # Return the text of the second non-empty <p> tag
  return np.nan  # Return NaN for non-string inputs and when the condition is not met

# Create a new column 'english_text' in the DataFrame and populate it with the extracted text
seals03['english_text'] = seals03['inscription_in_english'].apply(extract_second_p_text)

# Now the 'english_text' column contains the text of the second non-empty <p> tag for rows that meet the condition, and NaN for rows that do not


In [ ]:
#Getting the seals that arent formatted corretly so that I can fix them by hand
sealstest2 = seals03.drop(seals03.columns.difference(['inscription_in_english', 'english_text','location']), axis=1)
sealstest2 = sealstest2[sealstest2['english_text'].isna()]
sealstest2 = sealstest2[sealstest2['inscription_in_english'].notna()]
sealstest2.to_csv('sealstest2.csv')

In [ ]:
#Combining the subset and saving to a CSV
extrasealstest = pd.read_csv('/content/seals02englishtextfix.csv')
extrasealstest.set_index('Unnamed: 0', inplace=True)
extrasealstest.index.name = 'index'
seals03.index.name = 'index'
seals03['english_text'] = seals03['english_text'].combine_first(extrasealstest['english_text'])
#NEW CSV
seals03.to_csv('sealsdata03.csv', index=False)

## Messy, attempted creation of regex patterns. Because I decided to not go with this option this code is not documented

In [ ]:
# Function to extract English text using regular expressions
def extract_english(text):
    # Regular expression to match the first English word and capture everything after it
    english_pattern = r'[^A-Za-z\s]*\b(?!or\b)[A-Za-z]+.*'
    # Find the first match of the English pattern in the text
    english_match = re.search(english_pattern, text)
    # If match found, extract the text after the first English word
    return english_match.group(0).strip() if english_match else ''


# Apply the function to the column 'inscription_in_english'
df['english_text'] = df['inscription_in_english'].apply(extract_english)


In [ ]:
df[['inscription_in_english', 'english_text']].sample(n=100)

KeyError: "['english_text'] not in index"

In [ ]:

def contains_greek(text):
    # Regular expression to match only Greek characters excluding Latin characters
    greek_pattern = re.compile(r'[\u0391-\u03A9\u03B1-\u03C9]+')
    return bool(greek_pattern.search(text))


# Apply the function to the 'english_text' column
df['contains_greek'] = df['english_text'].apply(contains_greek)

# Filter the dataframe to show only rows where Greek characters are present
greek_rows = df[df['contains_greek'] == True]

# Print the rows with Greek characters
# Print the rows with non-English characters

In [ ]:
len(greek_rows['english_text'])

193

In [ ]:
greek_rows.drop(greek_rows.columns.difference(['english_text']), axis=1, inplace=True)

<ipython-input-56-f79ec5cf906e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  greek_rows.drop(greek_rows.columns.difference(['english_text']), axis=1, inplace=True)


In [ ]:
greek_rows

,english_text
152,"(sic) τῆς Μεγάλης Ἐκκλησίας. Lord, help your s..."
183,(Original strike) Εὐσταθ/μαθίου σφράγισμα . . ...
205,"O blessèd οne, protect the monk Makarios."
238,Seal of the imperial monastery of (St. John) P...
239,Seal of the imperial monastery of the Prodromo...
...,...
11899,Kωνσταντίνῳ βασιλικῷ σπαθαρίῳ καὶ κουράτορι. L...
12937,Kύριε βοήθει Γρηγορίῳ πρωτοσπαθαρίῳ καὶ πρωτον...
13411,"Bασιλείῳ τῷ Ἐνοικράτῃ. Lord, help your servant..."
13604,Bασίλειος ὁ Παξηνός. Basil Paxenos.


In [ ]:
wrong=greek_rows['english_text']

# Add century to the century column when century is in the seal title column but not century column

In [ ]:
seals03 = pd.read_csv('/content/sealsdata03.csv')
seals04 = seals03.copy()

In [ ]:
#testing regex pattern of the seal titles to see what it's not picking up on
a = seals04[(~seals04['seal_title'].str.contains(r'\(.*?centur(?:y|ies).*?\)')) &
            (~seals04['seal_title'].str.contains(r'\(.*?\d.*?\)')) &
            (seals04['century'].isna())]
a.head()

,Unnamed: 0,accession_number,seal_title,owner_name,owner_title,owner_office,location,institution_name,date_of_seal,century,...,related_items,seal_language,diameter_mm_min,diameter_mm_max,field_diameter_mm,weight_in_grams,relation_of_dies,physical_description,undertype,english_text
427,428,BZS.1955.1.5080,BZS.1955.1.5080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453,454,BZS.1955.1.5106,Blank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Greek,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,455,BZS.1955.1.5107,Blank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Greek,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,456,BZS.1955.1.5108,Blank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Greek,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,457,BZS.1955.1.5109,Blank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Greek,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Putting the century in the title into the same format it is in the 'century' column
def convert_ordinals(ordinals):
  '''list of ordinals(str) -> list of ordinals with digits(str)
  '''
  ordinal_mapping = {
      'fifth': '5th','sixth': '6th', 'seventh': '7th', 'eighth': '8th',
      'ninth': '9th', 'tenth': '10th', 'eleventh': '11th', 'twelfth': '12th',
      'thirteenth': '13th', 'fourteenth': '14th', 'fifteenth': '15th'
  }
  converted_ordinals = []
  for ordinal in ordinals:
      if ordinal in ordinal_mapping:
          converted_ordinals.append(ordinal_mapping[ordinal])
      else:
          converted_ordinals.append(ordinal)
  return converted_ordinals

#regex patterns used to extract the ordinals
iscentury = r'\([^\)\n]*?[^\)]centur(?:y|ies).*?\)'
numbercent = r'(?:fif|six|seven|eigh|nin|ten|eleven|twelf|thirteen|fourteen|fifteen)th'

def extract_century(text):
  '''str->str in format of digit ordinals seperated by '; ' '''
  if pd.isna(text) or not text:
    return
  if text == '':
    return
  parent = re.findall(iscentury,text,re.IGNORECASE)
  if parent:
    cent = sum([re.findall(numbercent, x, re.IGNORECASE) for x in parent],[])
    if cent:
      return '; '.join(convert_ordinals(cent))
  return

#testing with the subset of seals that the function would be applied to in the larger dataset
a = seals03[(seals03['seal_title'].str.contains(r'\([^\)\n]*?[^\)]centur(?:y|ies).*?\)')) &
            (seals03['century'].isna())]
a.loc[a['century'].isna(), 'century'] = a.loc[a['century'].isna(), 'seal_title'].apply(lambda x: extract_century(str(x)))
a[['seal_title','century']].head()

#applying to the full dataframe on rows where the century column is empty
seals04.loc[seals04['century'].isna(), 'century'] = seals04.loc[seals04['century'].isna(), 'seal_title'].apply(lambda x: extract_century(str(x)))

,seal_title,century
378,John proedros (= metropolitan) of Corfou (elev...,11th; 12th
403,The Virgin (named) Daphnin (eleventh/twelfth c...,11th; 12th
694,"Constantine Hagioprokopites, vestarches (eleve...",11th; 12th
1409,Niketas vestes and eidikos (eleventh century),11th
1422,Eustathios Markenos (eleventh century),11th


In [ ]:
#Converting to start and end years
def century_to_years(century):
  '''str-> int(year),int(year)
  '''
  if pd.isna(century):
    return None, None
  century = century.replace('th', '')
  centuries = century.split('; ')
  start_years = []
  end_years = []
  for c in centuries:
      c = int(c)
      start_year = (c - 1) * 100 + 1
      end_year = c * 100
      start_years.append(start_year)
      end_years.append(end_year)
  return int(min(start_years)), int(max(end_years))

#applying to the dataframe, creating new columns
seals04[['start_date', 'end_date']] = seals04['century'].apply(lambda x: pd.Series(century_to_years(x)))

In [ ]:
#Changing the columns into int64 so that the year can be int while still having NaN
seals04['start_date'] = seals04['start_date'].astype('Int64')
seals04['end_date'] = seals04['end_date'].astype('Int64')
seals04[['century','start_date', 'end_date']].head()

#NEW CSV
seals04.to_csv('sealsdata04.csv')

# Getting the titles and offices from the seal titles

In [ ]:
#split anything seperated by ;  into a list
def splitinto(text):
  '''text (delimited with ; ) -> list
  '''
  if isinstance(text, str):
      return text.split('; ')
  else:
      return []

In [ ]:
#subset of the dataframe
pos=seals04[['owner_office', 'owner_title','seal_title']]
#new rows splitting the offices and titles into lists
pos['officelist'] = pos['owner_office'].apply(splitinto)
pos['titlelist'] = pos['owner_title'].apply(splitinto)
#getting a list of the unique values in each
officelist = ((pos['officelist']).explode().unique())
officelist = [office for office in officelist if pd.notna(office)] #taking out NA
titlelist = ((pos['titlelist']).explode().unique())
titlelist = [title for title in titlelist if pd.notna(title)]
#adding spaces for the purpose of detectig them with a regex pattern
officelist1 = [f' {item} ' for item in officelist]
titlelist1 = [f' {item} ' for item in titlelist]

#excluding values that show up for both titles and offices because I don't want them to apply for both
#this eventually would need to be figured out or changed.
titlelist = [item for item in titlelist1 if item not in officelist1]
officelist = [item for item in officelist1 if item not in titlelist1]

#make sure that there are no overlapping items
set(officelist) & set(titlelist)

officelist.remove(' Emperor ')
officelist.remove(' Empress ')

<ipython-input-59-ee5f214f0122>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['officelist'] = pos['owner_office'].apply(splitinto)
<ipython-input-59-ee5f214f0122>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['titlelist'] = pos['owner_title'].apply(splitinto)


In [ ]:
#function to find items in a list and put them into a str list with a '; ' seperator
def extract_office_title(text, string_list):
  '''  str -> str with values seperated by '; '
  function to find items in a list and put them into a str list with a '; ' seperator
  '''
  text=text.lower()
  all_matches = []
  for item in string_list:
    item = item.lower()
    text.replace(",", " ")
    if item in text:
      all_matches.append(item)
  final_matches=all_matches.copy()
  for item in all_matches:
    #this part of the function checks if the item is part of a longer item that has also been selected
    #and if it is it chooses the longer one
    testlist=all_matches.copy()
    testlist.remove(item)
    for test in testlist:
      if (item in test):
       final_matches.remove(item)
       break
  if final_matches:
      return ';'.join(final_matches)[1:]  # Join multiple matches into a comma-separated string
  else:
      return

In [ ]:
def update_title(row,listex):
    if pd.isna(row['owner_title']):
        return extract_office_title(row['seal_title'],listex)
    else:
        return row['owner_title']

def update_office(row,listex):
    if pd.isna(row['owner_office']):
        return extract_office_title(row['seal_title'],listex)
    else:
        return row['owner_office']

# Update the 'owner_office' column
seals04['owner_office'] = seals04.apply(lambda row: update_office(row, officelist), axis=1)

# Update the 'owner_office' column
seals04['owner_title'] = seals04.apply(lambda row: update_title(row, titlelist), axis=1)

In [ ]:
#NEW CSV
seals04.to_csv('sealsdata05.csv',index=False)

In [ ]:
seals04 = pd.read_csv('/content/sealsdata05.csv')

seals04.drop(seals04.columns[[0, 1]], axis=1, inplace=True)

seals04['start_date'] = seals04['start_date'].astype('Int64')
seals04['end_date'] = seals04['end_date'].astype('Int64')

# Fixing accession numbers

In [ ]:
#Finding accession numbers in parallels and related items that don't have a match before the nodegoat import
seals04['parallels_split'] = seals04['parallels'].str.split('; ')
parallels_exploded = seals04.explode('parallels_split')
parallels_accessions = parallels_exploded['parallels_split'].unique()
# Find the accession numbers in 'parallels' that don't have a match in 'accession_number'
missing_accessions = [acc for acc in parallels_accessions if acc not in seals04['accession_number'].values]
missing_accessions
#I looked up all of these on the DO website and in the original CSV with different formatting (spaces, etc.)
#And I didn't find them. I just ran the nodegoat import with them in the original dataset and kept them as text
#So they are in nodegoat without connections

[nan,
 'BZS.1955.1.1326',
 'BZS.1955.1.1327',
 'BZS.1955.1.1036',
 'BZS.1955.1.1038']

In [ ]:
#This was fixing rows with accession numbers that don't match the pattern. If they contained extra characters it removes them, and then will show entries that still do not match the pattern.
testsubset2=seals04[~seals04['accession_number'].str.match(r'^BZS ?[.\d]+$', na=False)]
extracted = testsubset2['accession_number'].str.extract(r'^(BZS ?[.\d]+)', expand=False)
testsubset2.loc[extracted.notna(), 'accession_number'] = extracted
testsubset2[~testsubset2['accession_number'].str.match(r'^BZS ?[.\d]+$',
na=False)]

In [ ]:
# manually fixing accession numbers
seals04.loc[seals04.index[7759], 'accession_number'] = 'BZS.1955.1.3672'
seals04.loc[seals04.index[13049], 'accession_number'] = 'BZS.1958.106.4753'
seals04.loc[seals04.index[15067], 'accession_number'] = 'BZS.1955.1.2334'
seals04.loc[seals04.index[15125], 'accession_number'] = 'BZS.1955.1.2392'

# Truncating strings for nodegoat import

Nodegoat holds strings under 5k length, I just truncated them knowing there is easy access to the database with access to them.
(realizing later upon code documentation post project that the object type for commentary is text not string. I'm not sure if this is even nessecary.)

In [ ]:
#Making sure there are strings over 5k
seals04['length'] = seals04['commentary'].str.len()
seals04_ranked = seals04.sort_values(by='length', ascending=False)
print(seals04_ranked['length'].head())

In [ ]:
#making sure there are no other columns with long strings that need to be truncated
columns_with_long_strings = []
for col in seals04.columns:
    if seals04[col].apply(lambda x: len(str(x)) if isinstance(x, str) else 0).max() > 5000:
        columns_with_long_strings.append(col)
columns_with_long_strings

Columns with strings longer than 5000 characters:
[]


In [ ]:
#Truncating the strings
def truncate_long_strings(commentary):
  '''Str -> Str
  cuts any string over 5k chars at 4095
  '''
  if isinstance(commentary, str) and len(commentary) > 4095:
      return commentary[:4095]
  elif isinstance(commentary, str):
      return commentary
  else:
      return commentary

seals04['commentary'] = seals04['commentary'].apply(truncate_long_strings)



In [ ]:
#NEW CSV (This is the final CSV)
seals04.to_csv('sealsdata10.csv',index=False)

# Getting a list of dignities and titles in order to import them into nodegoat

In [ ]:
seals05 = pd.read_csv('/content/sealsdata10.csv')
seals05['start_date'] = seals05['start_date'].astype('Int64')
seals05['end_date'] = seals05['end_date'].astype('Int64')
seals05.drop('length', axis=1, inplace=True)

In [ ]:
seals05['owner_office']=seals05['owner_office'].str.lower()
seals05['owner_title']=seals05['owner_title'].str.lower()
officetiltes = seals05[~(seals05['owner_office'].isna() & seals05['owner_title'].isna())][['accession_number', 'owner_office', 'owner_title']]
officetiltes['officelist'] = officetiltes['owner_office'].apply(splitinto)
officetiltes['titlelist'] = officetiltes['owner_title'].apply(splitinto)
officelist = ((officetiltes['officelist']).explode().unique())
officelist = [office for office in officelist if pd.notna(office)]
titlelist = ((officetiltes['titlelist']).explode().unique())
titlelist = [title for title in titlelist if pd.notna(title)]

titledf=pd.DataFrame(titlelist)
titledf.to_csv("dignities.csv", index=False,header=False)
officedf=pd.DataFrame(officelist)
officedf.to_csv("offices.csv", index=False,header=False)
officetiltes.to_csv('offices_dignities_import.csv',index=False)

In [ ]:
officetiltes.to_csv('offices_dignities_import.csv',index=False)

In [ ]:
officetiltes['officelist'] = officetiltes['owner_office'].apply(splitinto)
officetiltes['titlelist'] = officetiltes['owner_title'].apply(splitinto)
officelist = ((officetiltes['officelist']).explode().unique())
officelist = [office for office in officelist if pd.notna(office)]
titlelist = ((officetiltes['titlelist']).explode().unique())
titlelist = [title for title in titlelist if pd.notna(title)]

# Notes to consider for using this data in a publication


The dataset probably needs to be sorted through- here are some of the errors

The accession numbers are not all formatted the same as some have spaces and they don't have the same number of digits. If the accession numbers get changed they also need to be changed in parallels and related items as they are consistent accross those columns

There are some offices/titles that are not properly applied to the office and title column but are in the seal title column:
1. Offices and titles that are both offices and titles at different points in time that are on the office/title list
2. Offices/titles that do not exist on those lists (offices that are in seal title or english text but are not acutally in those lists anywhere)

There are certain offices/titles that might have spelling problems or are inconsistent in english:
droungarious & droungarios
epiktes & epeiktes
are two examples of ones that look to be the same or could be the same in greek (unsure which is why I did't already touch them)


There are some repeat seals that have the same accession numbers

